In [43]:
import random 
import simpy 
import numpy as np

# Call center simulation

https://youtu.be/8SLk_uRRcgc 

In [44]:
# parameters for simulation
NUM_EMPLOYEES = 2
AVG_SUPPORT_TIME = 5
CUSTOMER_INTERVAL = 2
SIM_TIME = 120

# counter
customers_handled = 0

In [45]:
class CallCenter: 

    def __init__(self, env, num_employees, support_time): #env for simulation, from simpy
        self.env = env
        self.staff = simpy.Resource(env, num_employees) # set number of employees as a resource
        self.support_time = support_time

    def support(self, customer):
        random_time = max(1, np.random.normal(self.support_time, 4)) 
        yield self.env.timeout(random_time)
        print(f"Support fininshed for {customer} at {self.env.now:.2f}")

In [46]:
def customer(env, name, call_center):
    global customers_handled
    print(f"Customer {name} enters waiting queue at {env.now:.2f}")
    with call_center.staff.request() as request:
        yield request
        print(f"Customer {name} enters call at {env.now:.2f}")
        yield env.process(call_center.support(name))
        print(f"Customer {name} left call at {env.now:.2f}")
        customers_handled += 1

In [47]:
def setup(env, num_employees, support_time, customer_interval):
    call_center = CallCenter(env, num_employees, support_time)

    for i in range(1,6):
        env.process(customer(env, num_employees, call_center))

    while True: # while True is True --> loop forever 
        yield env.timeout(random.randint(customer_interval-1, customer_interval+1))
        i += 1
        env.process(customer(env, i, call_center))

In [48]:
print("Starting Call Center Simulation")
env = simpy.Environment()
env.process(setup(env, NUM_EMPLOYEES, AVG_SUPPORT_TIME, CUSTOMER_INTERVAL))
env.run(until=SIM_TIME)

print(f"Customer handled: {customers_handled}")

Starting Call Center Simulation
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters waiting queue at 0.00
Customer 2 enters call at 0.00
Customer 2 enters call at 0.00
Support fininshed for 2 at 1.00
Support fininshed for 2 at 1.00
Customer 2 left call at 1.00
Customer 2 left call at 1.00
Customer 2 enters call at 1.00
Customer 2 enters call at 1.00
Customer 6 enters waiting queue at 3.00
Support fininshed for 2 at 3.80
Customer 2 left call at 3.80
Customer 2 enters call at 3.80
Customer 7 enters waiting queue at 5.00
Support fininshed for 2 at 6.74
Customer 2 left call at 6.74
Customer 6 enters call at 6.74
Customer 8 enters waiting queue at 8.00
Support fininshed for 2 at 9.79
Customer 2 left call at 9.79
Customer 7 enters call at 9.79
Customer 9 enters waiting queue at 10.00
Customer 10 enters waiting queue at 11.00
Customer 11 enters waiting queue at 12.00
S

# Understanding generators

2 ways of creating generators:
1. Generator functions
2. Generator expressions

https://realpython.com/introduction-to-python-generators/

## Generator functions

Generator functions look and act just like regular functions, but with one defining characteristic. Generator functions use the Python yield keyword instead of return.

In [49]:
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1

inf = infinite_sequence()
print(next(inf))
print(next(inf))

0
1


`yield` indicates where a value is sent back to the caller, but unlike `return`, you *don’t exit the function afterward*.

Instead, the state of the function is *remembered*. That way, when next() is called on a generator object (either explicitly or implicitly within a for loop), the previously yielded variable num is incremented, and then yielded again

## Generator expressions

Like list comprehensions, generator expressions allow you to quickly create a generator object in just a few lines of code. They’re also useful in the same cases where list comprehensions are used, with an added benefit: you can create them without building and holding the entire object in memory before iteration. In other words, you’ll have *no memory penalty* when you use generator expressions

In [50]:
nums_squared_lc = [num**2 for num in range(5)] # creates list using brackets
nums_squared_gc = (num**2 for num in range(5)) # creates generator expression using parentheses

In [51]:
print(nums_squared_lc)
print(nums_squared_gc)

[0, 1, 4, 9, 16]
<generator object <genexpr> at 0x00000153FEDEFEB0>


# Event discrete simulation with SimPy

https://www.youtube.com/watch?v=Bk91DoAEcjY

In [52]:
def clock(env, name, tick):
    while True:
        # clock will repeat printing current simulation time
        print(name, env.now)
        # stop the clock
        yield env.timeout(tick)

# create environment
env = simpy.Environment() 

# print every half simulation step
env.process(clock(env, 'fast', 0.5))

# print every simulation step
env.process(clock(env, 'slow', 1))

# run until simulation time 2
env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


There are 2 kinds of environments:
1. Environment - simulate "as fast as possible"
2. RealtimeEnvionment - synchronized with wall-clock time
    * Observe real-time behaviour

In [53]:
# timeout events let time pass

def speaker(env, start):
    while True:
        until_start = start - env.now

        # let time pass to wait until the start of the talk
        yield env.timeout(until_start)

        # end the talk after 30 minutes
        yield env.timeout(30)

        # let next speaker start after prev speaker ends
        start = env.now

        # print end timings
        print(env.now)

env = simpy.Environment()
env.process(speaker(env, 10))
env.run(until=100)

40
70


In [55]:
# processes are events too

def speaker(env):
    # end the talk after 30 minutes
    yield env.timeout(30)
    return('end of talk', env.now)

def moderator(env):
    # 3 speakers
    for i in range(3):
        # spawn speaker process and wait for that to end before sending another speaker
        val = yield env.process(speaker(env))
        print(val)

env = simpy.Environment()
env.process(moderator(env))
env.run(until=100)

('end of talk', 30)
('end of talk', 60)
('end of talk', 90)


## Asynchronous interrupts

Documentation: https://simpy.readthedocs.io/en/latest/topical_guides/resources.html


In reality, not all speakers speak for exactly 30 minutes. Moderators must interrupt speakers if they exceed their time limit

In [79]:
from random import randint

def speaker(env):
    try:
        talk_duration = randint(25, 35)
        yield env.timeout(talk_duration)
        print(talk_duration)
    except simpy.Interrupt as interrupt:
        print(interrupt.cause)

def moderator(env):
    for i in range(3):
        speaker_proc = env.process(speaker(env))
        results = yield speaker_proc | env.timeout(30) 
        print(results)

        if speaker_proc not in results:
            speaker_proc.interrupt('No time left')

env = simpy.Environment()
env.process(moderator(env))
env.run()

<ConditionValue {<Timeout(30) object at 0x153fefcd940>: None}>
No time left
<ConditionValue {<Timeout(30) object at 0x153fefcdf70>: None}>
No time left
27
<ConditionValue {<Process(speaker) object at 0x153fefcd5b0>: None}>


## Shared resources 

Documentation: https://simpy.readthedocs.io/en/latest/topical_guides/resources.html

Shared resources are another way to model Process Interaction. They form a congestion point where processes *queue* up in order to use them.

3 categories:
1. Resource - Resources that can be used by a limited number of processes at a time (e.g., a gas station with a limited number of fuel pumps).
2. Store - Resources that allow the production and consumption of Python objects.
3. Container - Resources that model the production and consumption of a homogeneous, undifferentiated bulk. It may either be continuous (like water) or discrete (like apples).

All resources share the same basic concept: The resource itself is some kind of a container with a, usually limited, capacity. Processes can either try to put something into the resource or try to get something out. If the resource is full or empty, they have to queue up and wait.

In [ ]:
# imports
from random import randint 
import simpy

Now, we will use simpy to model conference attendees. We will model a continuous flow of talks sessions, with each session containing 3 talks of 30 mins each. There will be a 15 mins break between each session (3 talks)

In [80]:
# config
TALKS_PER_SESSION = 3
TALK_LENGTH = 30
BREAK_LENGTH = 15

In [ ]:
def attendee(env, name, knowledge=0, hunger=0):
    # repeat sessions
    while True:
        # visit talks
        for i in range(TALKS_PER_SESSION):
            knowledge += randint(0,3) / (1+hunger) # the more hungry the less knowledge
            hunger += randint(1,4)

            yield env.timeout(TALK_LENGTH)

        print(f'Attendee {name} finished talks with {knowledge:.2f} and {hunger}')